# Introduction

Here I'm trying out a QDN on the frozen lake problem. 

https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0


### Setup

In [38]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
%matplotlib inline

from keras.layers import InputLayer, Dense, Input
from keras.models import Model


class DQNAgent:
    def __init__(self, num_states, num_actions):
        
        self.num_states = num_states
        self.num_actions = num_actions
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._make_model()

        
    def _make_model(self):
        
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.num_states, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.num_actions, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        

    def act(self, state):
        
        #Epsilon greedy
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.num_actions)
        else:
            Q_values = self.model.predict(state)[0]
            return np.argmax(Q_values)
        

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                Qs_next_state = self.model.predict(next_state)[0]
                target = (reward + self.gamma*np.amax(Qs_next_state))
            target_Q = self.model.predict(state)   #as tensor
            target_Q[0][action] = target           #adjust only one of the values   
            self.model.fit(state, target_Q, epochs=1, verbose=0)
            
        #Freeze the greed
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            

    def vectorize_state(self,state):
        return np.identity(self.num_states)[state:state+1]
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
        
        
    def print_Qs(self):
        Qs = np.zeros((self.num_states, self.num_actions))
        for i,state in enumerate(range(self.num_states)):
            state_vector = self.vectorize_state(state)
            Qs_temp = self.model.predict(state_vector)[0]
            Qs[i] = Qs_temp
        return Qs

## Main

In [ ]:
import gym

#Environment
env = gym.make('FrozenLake-v0')
env.reset()
num_states = env.observation_space.n
num_actions = env.action_space.n  
num_episodes = 10**6
reward_list = []
batchsize = 32

#Agent
agent = DQNAgent(num_states, num_actions)  
var_s = []
agent.gamma = 0.99
agent.epsilon = 1.0

#Train
for episode in range(num_episodes):
    finished = False
    state_zero = 0
    state_vector = agent.vectorize_state(state_zero)
    reward_sum = 0
    while not finished:
        
        #Main loop
        action = agent.act(state_vector)
        next_state_scalar, reward, finished, _ = env.step(action)
        next_state_vector = agent.vectorize_state(next_state_scalar)
        agent.remember(state_vector,action,reward,next_state_vector,finished)
        state_vector = next_state_vector
        reward_sum += reward
        #agent.learn([state_vector,action,reward,next_state_vector])
        
        #Check learning, want to see if the NN is converging
        var = np.var(agent.model.get_weights()[0].flatten())   #variance in weights
        var_s.append(var)
                
    #Learn after each episode
    if len(agent.memory) >= batchsize:
        agent.replay(batchsize)
    #var = np.var(agent.model.get_weights()[0].flatten())   #variance in weights
    #var_s.append(var) 
    reward_list.append(reward_sum)
    env.reset()
    
#Print stats
print "Score over time: " +  str(sum(reward_list)/num_episodes)
plt.plot(reward_list)
print "Final Q-Table Values"
agent.print_Qs()

Finally! Getting performance comparable to the Table. 

### Search hyperparameters

In [35]:
def run(gamma,epsilon):
    
    #Environment
    env = gym.make('FrozenLake-v0')
    env.reset()
    num_states = env.observation_space.n
    num_actions = env.action_space.n  
    num_episodes = 1000
    reward_list = []
    batchsize = 32

    #Agent
    agent = DQNAgent(num_states, num_actions)
    agent.gamma = gamma
    agent.epsilon = epsilon

    #Train
    for episode in range(num_episodes):
        finished = False
        state_zero = 0
        state_vector = agent.vectorize_state(state_zero)
        reward_sum = 0
        while not finished:

            #Main loop
            action = agent.act(state_vector)
            next_state_scalar, reward, finished, _ = env.step(action)
            next_state_vector = agent.vectorize_state(next_state_scalar)
            agent.remember(state_vector,action,reward,next_state_vector,finished)
            state_vector = next_state_vector
            reward_sum += reward

        #Learn after each episode
        if len(agent.memory) >= batchsize:
            agent.replay(batchsize)
            
        reward_list.append(reward_sum)
        env.reset()

    #Print stats
    #print "Score over time: " +  str(sum(reward_list)/num_episodes)
    return reward_list

In [36]:
import time

gammas = [0.2,0.4,0.6,0.8,0.99]
eps = [0.2,0.4,0.6,0.8,0.99]

#gammas = [0.2,0.4]
#eps = [0.2, 0.4]

for i in range(len(gammas)):
    for j in range(len(eps)):
        gamma = gammas[i]
        epsilon = eps[j]
        t1 = time.time()
        reward_list = run(gamma, epsilon)
        t2 = time.time()
        print '(gamma, epsilon, score, T) = ' + str((gamma,epsilon,np.sum(reward_list) / 500.0, (t2-t1)/60.0))

(gamma, epsilon, score, T) = (0.2, 0.2, 0.066, 1.8004760185877482)
(gamma, epsilon, score, T) = (0.2, 0.4, 0.026, 2.009825666745504)
(gamma, epsilon, score, T) = (0.2, 0.6, 0.054, 1.8577500979105632)
(gamma, epsilon, score, T) = (0.2, 0.8, 0.056, 1.7253572662671408)
(gamma, epsilon, score, T) = (0.2, 0.99, 0.036, 1.7710787653923035)
(gamma, epsilon, score, T) = (0.4, 0.2, 0.104, 2.003510367870331)
(gamma, epsilon, score, T) = (0.4, 0.4, 0.076, 2.0297528823216755)
(gamma, epsilon, score, T) = (0.4, 0.6, 0.096, 2.074188601970673)
(gamma, epsilon, score, T) = (0.4, 0.8, 0.062, 2.129920752843221)
(gamma, epsilon, score, T) = (0.4, 0.99, 0.104, 2.022726368904114)
(gamma, epsilon, score, T) = (0.6, 0.2, 0.068, 2.2187981168429056)
(gamma, epsilon, score, T) = (0.6, 0.4, 0.122, 2.147476017475128)
(gamma, epsilon, score, T) = (0.6, 0.6, 0.092, 2.1772098819414776)
(gamma, epsilon, score, T) = (0.6, 0.8, 0.066, 2.214329131444295)
(gamma, epsilon, score, T) = (0.6, 0.99, 0.074, 1.9940408666928608)

Looks like big $\gamma$ is the best. And small epsilon. Although I think that depends more on the number of episodes.

### Baseline -- just using Q-tables

In [5]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .8
y = .95
num_episodes = 1000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)
    
print "Score over time: " +  str(sum(rList)/num_episodes)
print "Final Q-Table Values"
print Q

Score over time: 0.52
Final Q-Table Values
[[1.96317838e-01 2.52253700e-03 1.84336916e-02 1.65246608e-02]
 [3.33350057e-03 4.06910693e-03 2.67101255e-03 2.10738494e-01]
 [4.27830781e-03 3.55059227e-03 0.00000000e+00 1.07636921e-01]
 [7.02956830e-04 2.20111540e-03 2.15919483e-03 4.90879735e-02]
 [3.75676744e-01 3.25606239e-03 1.45667170e-03 1.67667806e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.13738664e-01 1.62768891e-05 1.43175308e-06 6.20232766e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.74443545e-05 6.03368308e-04 1.02493000e-02 3.15381353e-01]
 [6.17376276e-04 5.50168925e-01 2.78223213e-04 1.69717899e-03]
 [6.58548144e-01 1.96880508e-04 9.51677687e-04 4.16345690e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.43306984e-03 6.50613761e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.36245349e-01 0.00000000e+00]
 [0.00000000

Ok, so about half.

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)